In [2]:
!pip install huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.7 MB/s eta 0:00:00


In [7]:
!pip install safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 10.0 MB/s eta 0:00:0000:01


In [5]:
import importlib

required = [
    "torch", "torchvision", "torchaudio",
    "tqdm", "numpy", "soundfile", "librosa",
    "opencv-python", "decord", "imageio", "ftfy",
    "sentencepiece", "huggingface_hub", "safetensors", "timm", "einops"
]

for pkg in required:
    try:
        importlib.import_module(pkg.split('-')[0])  # 兼容opencv-python这种命名
        print(f" {pkg} 已安装")
    except ImportError:
        print(f" 缺少 {pkg}，请安装：pip install {pkg}")


 torch 已安装
 torchvision 已安装
 torchaudio 已安装
 tqdm 已安装
 numpy 已安装
 soundfile 已安装
 librosa 已安装
 缺少 opencv-python，请安装：pip install opencv-python
 缺少 decord，请安装：pip install decord
 缺少 imageio，请安装：pip install imageio
 ftfy 已安装
 缺少 sentencepiece，请安装：pip install sentencepiece
 huggingface_hub 已安装
 safetensors 已安装
 timm 已安装
 einops 已安装


In [ ]:
import os
import subprocess
from tqdm import tqdm

# =====================================================
# 路径配置
# =====================================================
BASE_PATH = "/workspace/dataset/IEMOCAP"
SESSION = "Session1"

session_path = os.path.join(BASE_PATH, SESSION, SESSION)
dialog_path = os.path.join(session_path, "dialog")
avi_path = os.path.join(dialog_path, "avi", "DivX")

print(f"Target session: {SESSION}")
print(f"AVI folder: {avi_path}")

# =====================================================
# 检查 ffmpeg 是否可用
# =====================================================
try:
    result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
    print("FFmpeg found:", result.stdout.split("\n")[0])
except Exception as e:
    print("FFmpeg not found or not executable:", e)
    raise SystemExit

# =====================================================
# 删除隐藏文件和
# =====================================================
def remove_hidden_and_ico_files(root_path):
    deleted = 0
    for root, _, files in os.walk(root_path):
        for f in files:
            if f.startswith(".") or f.startswith("._") or f.lower().endswith(".ico"):
                try:
                    os.remove(os.path.join(root, f))
                    deleted += 1
                except Exception as e:
                    print(f"Failed to delete {f}: {e}")
    return deleted

total_removed = remove_hidden_and_ico_files(avi_path)
print(f"Removed {total_removed} hidden/.ico files in {SESSION}")

# =====================================================
# 转换函数：.avi → .mp4（成功后自动删除 .avi）
# =====================================================
def convert_avi_to_mp4(avi_file):
    mp4_path = avi_file.replace(".avi", ".mp4")

    # 跳过已存在
    if os.path.exists(mp4_path):
        print(f"Skipping (already exists): {os.path.basename(mp4_path)}")
        return False

    cmd = [
        "ffmpeg", "-y", "-i", avi_file,
        "-c:v", "mpeg4",
        "-qscale:v", "3",
        "-pix_fmt", "yuv420p",
        "-threads", "1",
        "-loglevel", "error",
        mp4_path
    ]

    print(f"Converting: {os.path.basename(avi_file)} ...")
    try:
        subprocess.run(cmd, check=True)

        if os.path.exists(mp4_path) and os.path.getsize(mp4_path) > 1000000:  # 至少1MB
            print(f"Success: {os.path.basename(mp4_path)}")

            # 删除原始 .avi 文件
            try:
                os.remove(avi_file)
                print(f"Deleted original AVI: {os.path.basename(avi_file)}")
            except Exception as e:
                print(f"Failed to delete AVI: {avi_file}, reason: {e}")

            return True
        else:
            print(f"Conversion failed or output too small: {os.path.basename(mp4_path)}")
            return False
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error for {avi_file}: {e}")
        return False

# =====================================================
# 执行批量转码 + 删除
# =====================================================
if not os.path.exists(avi_path):
    print(f"AVI folder not found: {avi_path}")
else:
    avi_files = [f for f in os.listdir(avi_path) if f.endswith(".avi") and not f.startswith(".")]
    print(f"Found {len(avi_files)} .avi files to convert.\n")

    converted = 0
    deleted = 0

    for f in tqdm(avi_files, desc="Converting AVI → MP4"):
        full_path = os.path.join(avi_path, f)
        ok = convert_avi_to_mp4(full_path)
        if ok:
            converted += 1
            deleted += 1

    print("\n===========================")
    print("Conversion finished!")
    print(f"Converted: {converted}/{len(avi_files)}")
    print(f"AVI deleted: {deleted}")
    print("===========================\n")

print("Step 1 & 2 complete.")


Target session: Session1
AVI folder: /workspace/dataset/IEMOCAP/Session1/Session1/dialog/avi/DivX
FFmpeg found: ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
Removed 0 hidden/.ico files in Session1
Found 28 .avi files to convert.



Converting AVI → MP4:   0%|          | 0/28 [00:00<?, ?it/s]

Skipping (already exists): Ses01F_impro01.mp4
Skipping (already exists): Ses01F_impro02.mp4
Skipping (already exists): Ses01F_impro03.mp4
Skipping (already exists): Ses01F_impro04.mp4
Skipping (already exists): Ses01F_impro05.mp4
Skipping (already exists): Ses01F_impro06.mp4
Skipping (already exists): Ses01F_impro07.mp4
Skipping (already exists): Ses01F_script01_1.mp4
Skipping (already exists): Ses01F_script01_2.mp4
Skipping (already exists): Ses01F_script01_3.mp4
Skipping (already exists): Ses01F_script02_1.mp4
Skipping (already exists): Ses01F_script02_2.mp4
Skipping (already exists): Ses01F_script03_1.mp4
Skipping (already exists): Ses01F_script03_2.mp4
Skipping (already exists): Ses01M_impro01.mp4
Skipping (already exists): Ses01M_impro02.mp4
Skipping (already exists): Ses01M_impro03.mp4
Converting: Ses01M_impro04.avi ...


Converting AVI → MP4:  64%|██████▍   | 18/28 [00:05<00:03,  3.18it/s]

Success: Ses01M_impro04.mp4
Deleted original AVI: Ses01M_impro04.avi
Converting: Ses01M_impro05.avi ...


...

Success: Ses01M_script02_2.mp4
Deleted original AVI: Ses01M_script02_2.avi
Converting: Ses01M_script03_1.avi ...


Converting AVI → MP4:  96%|█████████▋| 27/28 [01:36<00:09,  9.38s/it]

Success: Ses01M_script03_1.mp4
Deleted original AVI: Ses01M_script03_1.avi
Converting: Ses01M_script03_2.avi ...


Converting AVI → MP4: 100%|██████████| 28/28 [01:46<00:00,  3.79s/it]

Success: Ses01M_script03_2.mp4
Deleted original AVI: Ses01M_script03_2.avi

Conversion finished!
Converted: 11/28
AVI deleted: 11

Step 1 & 2 complete.


In [5]:
# ============================================
#  IEMOCAP + ImageBind Multimodal Embedding Extraction
# ============================================

import os
import sys
import torch
from tqdm import tqdm
from imagebind.models import imagebind_model
from imagebind import data
import safetensors
from safetensors import torch as safetorch 
sys.modules["safetensors"] = safetensors




In [ ]:
import os, torch, gc, time
from tqdm import tqdm
from imagebind.models import imagebind_model
from safetensors.torch import load_file
from imagebind import data

# =====================================================
# 配置区域
# =====================================================
BASE_PATH = "/workspace/dataset/IEMOCAP"
OUTPUT_DIR = os.path.join(BASE_PATH, "embeddings")
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 模态开关（方便 debug）
USE_TEXT = True
USE_AUDIO = True
USE_VIDEO = True   

# =====================================================
# 显存监控与清理工具
# =====================================================
def gpu_mem():
    if torch.cuda.is_available():
        alloc = torch.cuda.memory_allocated() / 1024**3
        reserv = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU Memory - Allocated: {alloc:.2f} GB | Reserved: {reserv:.2f} GB")
    else:
        print("No CUDA available")

def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()
    time.sleep(0.5)
    gpu_mem()

# =====================================================
# 提取单样本 embedding（仅使用 mp4）
# =====================================================
def extract_embedding(text_path=None, audio_path=None, video_path=None):
    try:
        print("Starting embedding extraction...")
        gpu_mem()

        # ---- 加载模型 ----
        model = imagebind_model.imagebind_huge(pretrained=False).to(device)
        weight_path = "/ImageBind/model.safetensors"
        state_dict = load_file(weight_path, device=device)
        model.load_state_dict(state_dict, strict=False)
        model.eval()
        for p in model.parameters():
            p.requires_grad = False

        inputs = {}

        # ---- 文本 ----
        if USE_TEXT and text_path and os.path.exists(text_path):
            with open(text_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
            if text:
                inputs[imagebind_model.ModalityType.TEXT] = data.load_and_transform_text([text], device)

        # ---- 音频 ----
        if USE_AUDIO and audio_path and os.path.exists(audio_path):
            inputs[imagebind_model.ModalityType.AUDIO] = data.load_and_transform_audio_data([audio_path], device)

        # ---- 视频 (直接读 mp4，不再调用 ffmpeg) ----
        if USE_VIDEO and video_path and os.path.exists(video_path):
            try:
                inputs[imagebind_model.ModalityType.VISION] = data.load_and_transform_vision_data(
                    [video_path], device, num_frames=2
                )
            except Exception as e:
                print(f"Skipping video {video_path}: {e}")

        if not inputs:
            print("No valid inputs found for this sample.")
            del model
            clear_gpu()
            return None

        # ---- 提取特征 ----
        with torch.no_grad():
            embedding = model(inputs)

        # ---- 融合多模态 ----
        embed_list = [v for v in embedding.values()]
        fused = torch.stack(embed_list).mean(dim=0)

        # ---- 清理显存 ----
        del model, embedding, inputs
        clear_gpu()

        return fused.cpu()

    except Exception as e:
        print(f"Fatal error in extract_embedding: {e}")
        clear_gpu()
        return None


# =====================================================
# 文件过滤函数
# =====================================================
def is_valid_file(filename):
    return not (filename.startswith(".") or filename.startswith("._"))


# =====================================================
# 主循环：遍历所有 Session
# =====================================================
# sessions = [f"Session{i}" for i in range(1, 6)]
sessions = ["Session1"]

for session in sessions:
    print(f"\nProcessing {session} ...")
    clear_gpu()

    session_path = os.path.join(BASE_PATH, session, session)
    dialog_path = os.path.join(session_path, "dialog")

    wav_path = os.path.join(dialog_path, "wav")
    text_path = os.path.join(dialog_path, "transcriptions")
    avi_path = os.path.join(dialog_path, "avi", "DivX")  # 注意：这里仍是视频路径，只是文件改为 .mp4

    if not os.path.exists(wav_path):
        print(f"Missing wav directory in {session}")
        continue

    embeddings = {}

    for root, _, files in os.walk(wav_path):
        for file in tqdm(files, desc=f"{session} audio files"):
            if not is_valid_file(file) or not file.endswith(".wav"):
                continue

            base_id = os.path.splitext(file)[0]
            wav_file = os.path.join(root, file)
            txt_file = os.path.join(text_path, f"{base_id}.txt")
            mp4_file = os.path.join(avi_path, f"{base_id}.mp4")  # 直接读取 mp4 文件

            try:
                emb = extract_embedding(text_path=txt_file, audio_path=wav_file, video_path=mp4_file)
            except Exception as e:
                print(f"Error processing {base_id}: {e}")
                continue

            if emb is None:
                continue

            embeddings[base_id] = {
                "text_path": txt_file if os.path.exists(txt_file) else None,
                "audio_path": wav_file,
                "video_path": mp4_file if os.path.exists(mp4_file) else None,
                "embedding": emb.squeeze(0),
            }

            # 每5条保存一次临时结果
            if len(embeddings) % 5 == 0:
                temp_path = os.path.join(OUTPUT_DIR, f"{session}_partial.pt")
                torch.save(embeddings, temp_path)
                print(f"Saved checkpoint: {temp_path}")
                clear_gpu()

    out_path = os.path.join(OUTPUT_DIR, f"{session}_embeddings.pt")
    torch.save(embeddings, out_path)
    print(f"Saved {len(embeddings)} embeddings to {out_path}")

    del embeddings
    clear_gpu()

print("\nAll sessions processed successfully!")


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


Using device: cuda:0

Processing Session1 ...
GPU Memory - Allocated: 0.00 GB | Reserved: 0.00 GB


Session1 audio files:   0%|          | 0/28 [00:00<?, ?it/s]

Starting embedding extraction...
GPU Memory - Allocated: 0.00 GB | Reserved: 0.00 GB
Skipping video /workspace/dataset/IEMOCAP/Session1/Session1/dialog/avi/DivX/Ses01F_impro01.mp4: load_and_transform_vision_data() got an unexpected keyword argument 'num_frames'


In [2]:
# --------------------------------------------
# 基本路径设置
# --------------------------------------------
BASE_PATH = "/workspace/dataset/IEMOCAP"
OUTPUT_DIR = os.path.join(BASE_PATH, "embeddings")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------------------------
# 设备配置
# --------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

# --------------------------------------------
# 加载 ImageBind 模型
# --------------------------------------------
# 1️ 初始化空模型结构
model = imagebind_model.imagebind_huge(pretrained=False)


# 2️ 加载权重（直接读取 safetensors 文件）
weight_path = "/ImageBind/model.safetensors"  #  safetensors 文件路径
state_dict = safetorch.load_file(weight_path) 

# 3️ 将权重加载进模型
missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)

# 打印加载情况
if missing_keys:
    print(f"  Missing keys: {len(missing_keys)} (部分权重可能未匹配)")
if unexpected_keys:
    print(f"  Unexpected keys: {len(unexpected_keys)} (模型结构有少量差异)")
if not missing_keys and not unexpected_keys:
    print("  All model weights loaded successfully!")

model.eval().to(device)
print(" ImageBind model loaded successfully!")

# 冻结所有参数
for param in model.parameters():
    param.requires_grad = False

print("All ImageBind parameters are frozen.")

# --------------------------------------------
# 文件过滤函数：跳过所有 macOS 垃圾文件
# --------------------------------------------
def is_valid_file(filename):
    if filename.startswith(".") or filename.startswith("._"):
        return False
    return True



 Using device: cuda:0
  All model weights loaded successfully!
 ImageBind model loaded successfully!
All ImageBind parameters are frozen.


In [25]:
print("Notebook 仍在运行，输出刷新正常")


Notebook 仍在运行，输出刷新正常


In [3]:
import os, torch, gc
from tqdm import tqdm
from imagebind.models import imagebind_model
from imagebind import data


# --------------------------------------------
# 提取单个样本 embedding（模型即时加载/卸载）
# --------------------------------------------
def extract_embedding(text_path=None, audio_path=None, video_path=None):
    #  每次都加载模型（不会占用太久显存）
    model = imagebind_model.imagebind_huge(pretrained=False).to(device)
    weight_path = "/ImageBind/model.safetensors"
    state_dict = torch.load(weight_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    for p in model.parameters():
        p.requires_grad = False
        
    inputs = {}

    if text_path and os.path.exists(text_path):
        with open(text_path, "r", encoding="utf-8") as f:
            text = f.read().strip()
        if len(text) > 0:
            inputs[imagebind_model.ModalityType.TEXT] = data.load_and_transform_text([text], device)

    if audio_path and os.path.exists(audio_path):
        inputs[imagebind_model.ModalityType.AUDIO] = data.load_and_transform_audio_data([audio_path], device)

    if video_path and os.path.exists(video_path):
        inputs[imagebind_model.ModalityType.VISION] = data.load_and_transform_vision_data(
            [video_path], device, num_frames=2
        )

    if len(inputs) == 0:
        del model
        torch.cuda.empty_cache()
        gc.collect()
        return None

    with torch.no_grad():
        embedding = model(inputs)

    embed_list = [v for v in embedding.values()]
    fused = torch.stack(embed_list).mean(dim=0)

    #  清理显存
    del model, embedding, inputs
    torch.cuda.empty_cache()
    gc.collect()

    return fused.cpu()


# --------------------------------------------
# 遍历每个 Session
# --------------------------------------------
sessions = [f"Session{i}" for i in range(1, 6)]

for session in sessions:
    print(f"\n Processing {session} ...")
    torch.cuda.empty_cache()
    gc.collect()

    session_path = os.path.join(BASE_PATH, session, session)
    dialog_path = os.path.join(session_path, "dialog")

    wav_path = os.path.join(dialog_path, "wav")
    text_path = os.path.join(dialog_path, "transcriptions")
    avi_path = os.path.join(dialog_path, "avi", "DIVX")

    if not os.path.exists(wav_path):
        print(f" Missing wav directory in {session}")
        continue

    embeddings = {}

    for root, _, files in os.walk(wav_path):
        for file in tqdm(files, desc=f"{session} audio files"):
            if not is_valid_file(file) or not file.endswith(".wav"):
                continue

            base_id = os.path.splitext(file)[0]
            wav_file = os.path.join(root, file)
            txt_file = os.path.join(text_path, f"{base_id}.txt")
            avi_file = os.path.join(avi_path, f"{base_id}.avi")

            try:
                emb = extract_embedding(text_path=txt_file, audio_path=wav_file, video_path=avi_file)
            except Exception as e:
                print(f" Error processing {base_id}: {e}")
                continue

            if emb is None:
                continue

            embeddings[base_id] = {
                "text_path": txt_file if os.path.exists(txt_file) else None,
                "audio_path": wav_file,
                "video_path": avi_file if os.path.exists(avi_file) else None,
                "embedding": emb.squeeze(0),
            }

            #  每 5 条保存一次中间结果
            if len(embeddings) % 5 == 0:
                torch.save(embeddings, os.path.join(OUTPUT_DIR, f"{session}_partial.pt"))
                torch.cuda.empty_cache()
                gc.collect()

    out_path = os.path.join(OUTPUT_DIR, f"{session}_embeddings.pt")
    torch.save(embeddings, out_path)
    print(f" Saved {len(embeddings)} embeddings to {out_path}")

    del embeddings
    torch.cuda.empty_cache()
    gc.collect()

print("\n All sessions processed successfully!")



 Processing Session1 ...


Session1 audio files:  52%|█████▏    | 29/56 [00:11<00:11,  2.44it/s]

 Error processing Ses01F_impro01: could not find MARK


Session1 audio files:  54%|█████▎    | 30/56 [00:22<00:23,  1.11it/s]

 Error processing Ses01F_impro02: could not find MARK


...

 Error processing Ses01M_script02_1: could not find MARK


Session1 audio files:  96%|█████████▋| 54/56 [04:27<00:20, 10.04s/it]

 Error processing Ses01M_script02_2: could not find MARK


Session1 audio files:  96%|█████████▋| 54/56 [04:36<00:10,  5.11s/it]


KeyboardInterrupt: 